# Text Generation

Finally, after our data cleaning and analysis we have reached the step where our goal is about: Generate motivational quotes.

We will use a tool that gives a basic text generation: Markov Chains (https://en.wikipedia.org/wiki/Markov_chain)

It is not as good as Deep Learning but it is a good starting point.

In [1]:
import pickle
filename = "corpus.pkl"
data = []
outfile = open(filename,'rb')
corpus = pickle.load(outfile)
text = ' '.join(corpus)

In [2]:
from collections import defaultdict

def markov_chain(text):
    
    # Tokenize the text by word, though including punctuation
    words = text.split(' ')
    
    # Initialize a default dictionary to hold all of the words and next words
    m_dict = defaultdict(list)
    
    # Create a zipped list of all of the word pairs and put them in word: list of next words format
    for current_word, next_word in zip(words[0:-1], words[1:]):
        m_dict[current_word].append(next_word)

    # Convert the default dict back into a dictionary
    m_dict = dict(m_dict)
    return m_dict

In [3]:
quotes = markov_chain(text)
quotes

{'all': ['our',
  'the',
  'day',
  'means',
  'the',
  'the',
  'sunshine',
  'the',
  'we',
  'of',
  'the',
  'success',
  'your',
  'this',
  '–',
  'within',
  'the',
  'or',
  'at',
  'my'],
 'our': ['dreams', 'voices', 'lives', 'lives', 'greatest'],
 'dreams': ['can',
  'and',
  'hustlers',
  'dont',
  'than',
  'dont',
  'and',
  'come',
  'read',
  'are',
  'dont',
  'revenge'],
 'can': ['come',
  'i',
  'go',
  'be',
  'imagine',
  'either',
  'do',
  'let',
  'waste',
  'shake',
  'go',
  'do',
  'cry',
  'dim',
  'do',
  'with',
  'dream',
  'do',
  'do',
  'do',
  'do',
  'do',
  'do',
  'do',
  'start',
  'and',
  'be',
  'do',
  'hurt',
  'fill',
  'and',
  'make',
  'see',
  'see',
  'control',
  'use',
  'change',
  'make',
  'make',
  'only',
  'change',
  'change',
  'change',
  'endure',
  'whatever',
  'change',
  'use',
  'do',
  'go',
  'go',
  'inspire',
  'provoke'],
 'come': ['true', 'to', 'to', 'everyone', 'true', 'true', 'from'],
 'true': ['if', 'not', 'that

In [65]:
import random

def generate_sentence(chain, count=4):
    '''Input a dictionary in the format of key = current word, value = list of next words
       along with the number of words you would like to see in your generated sentence.'''

    # Capitalize the first word
    word1 = random.choice(list(chain.keys()))
    sentence = word1.capitalize()

    # Generate the second word from the value list. Set the new word as the first word. Repeat.
    for i in range(count-1):
        word2 = random.choice(chain[word1])
        word1 = word2
        sentence += ' ' + word2

    # End it with a period
    sentence += '.'
    return(sentence)

In [137]:
generate_sentence(quotes)

'Rather become a dreamer.'

Some quotes:

"Lesson in yourself the key to and hard work ethic"

'24 hours a difference between successful people are a monday.'

'Bicycle youre the time trying to be perfectly happy with.'

'Know we wont always wake.'

'Late to wait a little.'

'Dawn its not something new.'

'Answers do it takes sweat.'

'Forget your excuses the whole.'

'Glory is the middle.'

'Achieve it takes nothing.'

'Selfconfidence make a failure.'

'Rather become a dreamer.'

This one is funny...

'Confidence and expect to build the capacity to blame for.'

Ok...
 
'Werent paying attention to be.'

In [114]:
import tweepy
import logging
import os

logger = logging.getLogger()

def create_api():
    consumer_key = "tnxXtRfP9gIgwqLPPhB2gHDsm"
    consumer_secret = "DArsX9Gh00AqjPwW2Nrdb7gsI8uxJvTJ3ylWBUcyCaetcJgXfJ"
    access_token = "348194741-HZDY52bCXFtfbF9qwZIH7wH09Zvgqf2GFeUwZy2O"
    access_token_secret = "Ak7F0b0IyJwnwIfvkUi4PU6wK1xRo1mkhHgkB1XwXdMz8"

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, 
        wait_on_rate_limit_notify=True)
    try:
        api.verify_credentials()
    except Exception as e:
        logger.error("Error creating API", exc_info=True)
        raise e
    print("API created")
    return api

In [115]:
create_api()

API created


In [150]:
import time

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

def check_mentions(api, keywords, since_id):
    logger.info("Retrieving mentions")
    new_since_id = since_id
    for tweet in tweepy.Cursor(api.mentions_timeline,
        since_id=since_id).items():
        new_since_id = max(tweet.id, new_since_id)
        if tweet.in_reply_to_status_id is not None:
            continue
        if any(keyword in tweet.text.lower() for keyword in keywords):
            logger.info("Generating the quote...")

            api.update_status(
                status="Hey there! Here it goes: \'"+str(generate_sentence(quotes))+"\'",
                in_reply_to_status_id=tweet.id,
            )
    return new_since_id

In [ ]:
def main():
    api = create_api()
    since_id = 1
    while True:
        since_id = check_mentions(api, ["share a quote"], since_id)
        logger.info("Waiting...")
        time.sleep(20)

if __name__ == "__main__":
    main()

INFO:root:Retrieving mentions


API created


INFO:root:Generating the quote...
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentio